In [1]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn
from torch import autograd
import torch.optim as optim
import torch.nn.functional as F
import math
from torch.autograd import Variable
import copy
import pandas
import Queue
# import pyedflib
from os import walk
from sklearn.model_selection import train_test_split
import gc

In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6199917269217241025
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10741206221
locality {
  bus_id: 1
}
incarnation: 6036455015682469890
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10972997223
locality {
  bus_id: 1
}
incarnation: 15832592981884255668
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"
]


In [2]:
dir = "np_data13"
files = []
data_set_t = []
target_t = []

for (dirpath, dirnames, filenames) in walk(dir):
    files.extend(filenames)
    break
for instance, file  in enumerate(files):
    if (file == "T0.npy" or file == "T1.npy" or file == "T2.npy" or file == "T3.npy" or file == "T4.npy" or file == "T5.npy"):
        continue
    if (file == "0.npy" or file == "1.npy" or file == "2.npy" or file == "3.npy" or file == "4.npy" or file == "5.npy"):
        data_set_t.append(np.load(dir + '/' +file))
        target_t.append(np.load(dir + '/T' +file))
calm_test= np.load(dir + '/calm_test.npy')
calm_train = np.load(dir + '/calm_train.npy')

Tcalm_test= np.load(dir + '/Tcalm_test.npy')
Tcalm_train = np.load(dir + '/Tcalm_train.npy')

data_nsh = np.vstack((data_set_t[0],data_set_t[1],data_set_t[2],data_set_t[3],data_set_t[4], data_set_t[5]))
targets_nsh = np.hstack((target_t[0],target_t[1],target_t[2],target_t[3],target_t[4],  target_t[5]))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data_nsh, targets_nsh, test_size=0.03, shuffle = True)

data_set = np.vstack((X_train, calm_train))
targets = np.hstack((y_train, Tcalm_train))
# data_set = np.transpose(data_set, (0, 2, 1))

val_data = np.vstack((calm_test[120:248], X_test))
val_targs = np.hstack((Tcalm_test[120:248], y_test))
# val_data = np.transpose(val_data, (0, 2, 1))

In [4]:
del X_train
del X_test
del y_train
del y_test
del data_nsh
del targets_nsh
gc.collect()
del calm_test
del calm_train
del Tcalm_test
del Tcalm_train

In [5]:
def get_data(dataset, targets, batch_size):
    indexes = np.arange(dataset.shape[0])
    np.random.shuffle(indexes)
    
    dataset = dataset[indexes]
    targets = targets[indexes]
    
    tr_data = np.zeros((dataset.shape[0]/batch_size, batch_size, 31, 500))   
    tr_targs = np.zeros((dataset.shape[0]/batch_size, batch_size))
    
    for i in range (dataset.shape[0]/batch_size):
        tr_data[i] = copy.deepcopy(dataset[i*batch_size:(i+1)*batch_size])
        tr_targs[i] = copy.deepcopy(targets[i*batch_size: (i+1)*batch_size])
    return tr_data, tr_targs

In [565]:
class LikeDislikeModel(nn.Module):
    def __init__(self, 
                 in_channels,
                 out_size,
                 L_sizes = [300, 200, 300],
                 net = None,
                 fc_sizes = [700, 250],
                 n_local_pred = 3,
                 f_sizes = [3, 3, 5], 
                 channels = [256, 128, 64], 
                 strides = [2, 2, 3], 
                 use_cuda = True):
        
        super(LikeDislikeModel, self).__init__()
        
        self.in_channels = in_channels

        self.fc_sizes = fc_sizes
        self.out_size = out_size
        self.n_local_pred = n_local_pred
        self.use_cuda = use_cuda
        self.L_sizes = L_sizes
        self.net = net
        
        self.f_sizes = f_sizes
        self.channels = channels
        self.strides = strides
        self.len_sizes = self._len_sizes()
        
        self.conv1 = nn.Conv1d(in_channels, channels[0], f_sizes[0], strides[0])
        self.conv2 = nn.Conv1d(channels[0], channels[1], f_sizes[1], strides[1])              
        self.conv3 = nn.Conv1d(channels[1], channels[2], f_sizes[2], strides[2])

        self.fc1 = nn.Linear(self.len_sizes[-1] * self.channels[-1], fc_sizes[0])
        self.fc2 = nn.Linear(fc_sizes[0], fc_sizes[1])
        self.fc3 = nn.Linear(fc_sizes[1], out_size)
        
        self.batchnorm = nn.ModuleList()
        for convNum in range(3):
            self.batchnorm.append(nn.BatchNorm1d(channels[convNum]))
            
        self.dropout = nn.Dropout(p=0.5)
        
    def _len_sizes(self):
        len_sizes = []

        temp = []
        for j in range (len(self.f_sizes)):
            if j == 0:
                temp.append(int(math.floor((self.L_sizes - self.f_sizes[j])/self.strides[j] + 1)))
            else:
                temp.append(int(math.floor((temp[j-1] - self.f_sizes[j])/self.strides[j] + 1)))

        
        return temp

    def preprocessing(self, input_batch):
        #zero_mean = input_batch - input_batch.mean()
        """     
        inp = np.zeros((input_batch.shape))
        for n in range(input_batch.shape[0]):
            zero_mean = input_batch[n] - input_batch[n].mean()
            
            inp[n] = zero_mean/(np.std(zero_mean,axis = 0))
        del input_batch
        del zero_mean"""
        
        if self.use_cuda:
            input_batch = Variable(torch.FloatTensor(input_batch)).cuda()
        else:
            input_batch = Variable(torch.FloatTensor(input_batch))
        #print (input_batch[0])
        return input_batch

    def createLocalNetworks(self, nn):

        conv1 = self.conv1(nn)
        bn1 = self.batchnorm[0](conv1)
        #print (conv1)
        relu1 = F.relu(bn1)

        conv2 = self.conv2(relu1)
        bn2 = self.batchnorm[1](conv2)
        relu2 = F.relu(bn2)

        conv3 = self.conv3(relu2)
        bn3 = self.batchnorm[2](conv3)
        relu3 = F.relu(bn3)

        relu3 = relu3.view(self.batch_size, -1)

        fc1 = self.fc1(relu3)
        relu4 = F.relu(fc1)
        do1 = self.dropout(relu4)

        fc2 = self.fc2(do1)
        relu5 = F.relu(fc2)            

        do2 = self.dropout(relu5)

        fc3 = self.fc3(do2)
            
        return fc3
    
    def net_pred(self, nn):
        
        conv1 = self.conv1(nn)
        bn1 = self.batchnorm[0](conv1)
        relu1 = F.relu(bn1)

        conv2 = self.conv2(relu1)
        bn2 = self.batchnorm[1](conv2)
        relu2 = F.relu(bn2)

        conv3 = self.conv3(relu2)
        bn3 = self.batchnorm[2](conv3)
        relu3 = F.relu(bn3)

        relu3 = relu3.view(self.batch_size, -1)

        fc1 = self.fc1(relu3)
        relu4 = F.relu(fc1)
        
        
        fc2 = self.fc2(relu4)
        relu5 = F.relu(fc2)            

        fc3 = self.fc3(relu5)
            
        return fc3
    
    def _loss(self, raw_output, targets):
        
        if self.use_cuda:
            targets = Variable(torch.LongTensor(targets)).cuda()
        else:
            targets = Variable(torch.LongTensor(targets))
            
        loss = F.cross_entropy(raw_output, targets)
        return loss
    
    def accurancy(self, input_batch, targets):
        self.batch_size = input_batch.shape[0]
        nns  = self.preprocessing(input_batch)
        raw_output = self.net_pred(nns)
        #raw_output = self.createLocalNetworks(input_batch)
        
        _, index = torch.max(raw_output,1)
        index = index.cpu().data.numpy()
        acc = (index == targets).mean()
        return acc
    
    def forward(self, input_batch, targets):
        self.batch_size = input_batch.shape[0]
        nns  = self.preprocessing(input_batch)
        raw_output = self.createLocalNetworks(nns)
        #raw_output = self.createLocalNetworks(input_batch)
        loss = self._loss(raw_output, targets)
        return loss
   
    def predict(self, input_batch):
        self.batch_size = input_batch.shape[0]
        nns  = self.preprocessing(input_batch)
        raw_output = self.net_pred(nns)
        #raw_output = self.createLocalNetworks(input_batch)
            
        _, index = torch.max(raw_output,1)
        index = index.cpu().data.numpy()
        return index

In [567]:
logdir_root = './logdir2'
model = LikeDislikeModel(31, 3, L_sizes = 500,
                 net = 3,
                 fc_sizes = [7500, 1500],
                 n_local_pred = 1,
                 f_sizes = [3, 3, 5], 
                 channels = [256, 128, 64], 
                 strides = [2, 2, 3], 
                 use_cuda = True)
if model.use_cuda:
    model.cuda()
model.load_state_dict(torch.load(logdir_root + '/block0-loss=0.632_model.txt'))
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay = 0.03)

In [535]:
model.forward(val_data, val_targs)

Variable containing:
-2.1229e-06 -2.2399e-06 -2.3691e-06  ...  -1.4273e-06 -1.5577e-06 -1.7114e-06
-2.7088e-06 -2.8214e-06 -2.9426e-06  ...  -2.9461e-06 -2.8583e-06 -2.7758e-06
-5.0312e-06 -5.1097e-06 -5.1855e-06  ...  -3.9228e-06 -4.2068e-06 -4.5097e-06
                ...                   ⋱                   ...                
 1.4884e-07  5.2590e-08 -4.8773e-08  ...   4.6439e-07  4.8611e-07  4.9790e-07
-3.0315e-06 -3.1530e-06 -3.2895e-06  ...  -5.2069e-07 -1.2924e-07  2.2967e-07
-2.5119e-06 -2.3377e-06 -2.1838e-06  ...   2.3019e-06  2.9117e-06  3.4292e-06
[torch.cuda.FloatTensor of size 31x500 (GPU 0)]

Variable containing:
( 0 ,.,.) = 
  4.5850e-02  4.5849e-02  4.5849e-02  ...   4.5848e-02  4.5847e-02  4.5847e-02
  6.1525e-02  6.1525e-02  6.1526e-02  ...   6.1526e-02  6.1526e-02  6.1526e-02
 -5.8780e-02 -5.8780e-02 -5.8780e-02  ...  -5.8781e-02 -5.8781e-02 -5.8781e-02
                 ...                   ⋱                   ...                
  7.1091e-02  7.1091e-02  7.1091e-

Variable containing:
 0.5351
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [561]:
print( model.predict(val_data[0:128]))
print( model.predict(val_data))
"""print (val_data.shape)

for i in range (300):
    print (model.predict(val_data[i].reshape(1, 31, 500)))"""

[2 1 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 2 2 2 2 2 2
 2 2 2 1 2 1 2 2 1 2 2 1 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 1 1 2 2 0 2 2 1 1 2 2 2 2 2 2 2
 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 2 2 2 1 2 0 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 0 2 2 2
 2 0 2 2 2 2 2 2 2 2 2 2 2 2 0 0 2 2 2 2 2 2 2 1 0 2 2 2 2 2 2 2 2 2 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 1 2 2 2 2 2 1 2
 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

'print (val_data.shape)\n\nfor i in range (300):\n    print (model.predict(val_data[i].reshape(1, 31, 500)))'

In [15]:
val_data.shape

(387, 31, 500)

In [506]:
model.predict(val_data[0].reshape(1,31,500))

Variable containing:
-2.1229e-06 -2.2399e-06 -2.3691e-06  ...  -1.4273e-06 -1.5577e-06 -1.7114e-06
-2.7088e-06 -2.8214e-06 -2.9426e-06  ...  -2.9461e-06 -2.8583e-06 -2.7758e-06
-5.0312e-06 -5.1097e-06 -5.1855e-06  ...  -3.9228e-06 -4.2068e-06 -4.5097e-06
                ...                   ⋱                   ...                
 1.4884e-07  5.2590e-08 -4.8773e-08  ...   4.6439e-07  4.8611e-07  4.9790e-07
-3.0315e-06 -3.1530e-06 -3.2895e-06  ...  -5.2069e-07 -1.2924e-07  2.2967e-07
-2.5119e-06 -2.3377e-06 -2.1838e-06  ...   2.3019e-06  2.9117e-06  3.4292e-06
[torch.cuda.FloatTensor of size 31x500 (GPU 0)]



array([2])

In [ ]:
a == b

In [568]:
a = model.predict(val_data[:250])

torch.Size([250, 31, 500])
torch.Size([250, 256, 249])


In [361]:
model.accurancy(val_data, val_targs)
#val_targs

0.67183462532299743

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.000006, weight_decay = 0)

In [549]:
logdir_root = './logdir2'
#ValLossFile = open(logdir_root+'/val_loss.txt', 'w')

iterations = data_set.shape[0] / 256
epochs = 1000

#log_loss = np.zeros(epochs*iterations)
block = 0

for epoch in range(epochs):
    tr_data, tr_targs = get_data(data_set, targets, 256)
    
    for iteration in range(iterations):
        batch = tr_data[iteration]
        b_targets = tr_targs[iteration]
        
        model.zero_grad()
        loss = model(batch, b_targets)
        #log_loss[epoch*iterations + iteration] = loss.data
        loss.backward()
        optimizer.step()
    
    #if epoch % 10 == 0:
    
    if model.use_cuda:
        v_acc = model.accurancy(val_data, val_targs)
        acc =  model.accurancy(batch, b_targets)
        v_loss = model(val_data, val_targs)
        
        print('epoch {:d}, loss = {:.3f}, val_loss = {:.3f}, val_acc = {:.3f}, train_acc = {:.3f}'
              .format(epoch, loss.cpu().data[0], v_loss.cpu().data[0], v_acc, acc))
        #ModelFile = open(logdir_root+'/block{:d}-loss={:.3f}_model.txt'.format(block, loss.cpu().data[0]), 'w')
        #ValLossFile.write('{:.3f} {:.3f}\n'.format(acc, v_acc))
    else:
        print('block {:d}, loss = {:.3f}'
              .format(block, loss.data[0]))
        ModelFile = open(logdir_root+'/block{:d}-loss={:.3f}_model.txt'.format(block, loss.data[0]), 'w')
        #ValLossFile.write('{:.3f} {:.3f}\n'.format(loss.data[0], v_loss.data[0]))
   # block += 1

    #torch.save(model.state_dict(), ModelFile)
    #ModelFile.close()
#np.savetxt('log_loss.txt', log_loss, fmt='%1.3f')

epoch 0, loss = 1.097, val_loss = 1.098, val_acc = 0.354, train_acc = 0.387
epoch 1, loss = 1.098, val_loss = 1.098, val_acc = 0.354, train_acc = 0.355
epoch 2, loss = 1.091, val_loss = 1.098, val_acc = 0.354, train_acc = 0.453
epoch 3, loss = 1.093, val_loss = 1.099, val_acc = 0.354, train_acc = 0.395
epoch 4, loss = 1.097, val_loss = 1.099, val_acc = 0.354, train_acc = 0.367
epoch 5, loss = 1.093, val_loss = 1.099, val_acc = 0.354, train_acc = 0.371
epoch 6, loss = 1.093, val_loss = 1.099, val_acc = 0.354, train_acc = 0.367
epoch 7, loss = 1.094, val_loss = 1.099, val_acc = 0.354, train_acc = 0.402
epoch 8, loss = 1.094, val_loss = 1.100, val_acc = 0.354, train_acc = 0.312
epoch 9, loss = 1.088, val_loss = 1.099, val_acc = 0.354, train_acc = 0.438
epoch 10, loss = 1.087, val_loss = 1.099, val_acc = 0.354, train_acc = 0.438
epoch 11, loss = 1.092, val_loss = 1.100, val_acc = 0.354, train_acc = 0.387
epoch 12, loss = 1.093, val_loss = 1.100, val_acc = 0.354, train_acc = 0.395
epoch 13,

KeyboardInterrupt: 

In [13]:
np.savetxt('log_loss.txt', log_loss, fmt='%1.3f')

In [24]:
tst_data.shape

(179, 62, 1200)

In [70]:
test = np.zeros((256, 62, 1200))
for i in range (tst_data.shape[0]):
    test[i] = tst_data[i]

In [44]:
nns = model.preprocessing(batch)
raw_output = model.createLocalNetworks(nns)
scores = Variable(torch.FloatTensor(model.batch_size, model.out_size).zero_()).cuda()
for nn_num in range( model.n_local_pred):
    scores = scores + raw_output[nn_num]

In [45]:
#scores = scores[:tst_data.shape[0]]

print (scores.shape)
_, index = torch.max(scores,1)
index = index.cpu().data.numpy()
print ((index == b_targets).mean())

torch.Size([256, 3])
0.53125


In [32]:
print (model.accurancy(np.transpose(calm_test[400:600],(0,2,1)),np.zeros((200))))

0.02


In [160]:
ModelFile = open(logdir_root+'/block{:d}-loss={:.3f}_model.txt'.format(block, loss.cpu().data[0]), 'w')
torch.save(model.state_dict(), ModelFile)
ModelFile.close()

SyntaxError: invalid syntax (<ipython-input-3-ae6f5027f0d6>, line 1)

In [2]:
from keras import backend as K

ImportError: cannot import name np_utils

In [1]:
import keras

Using TensorFlow backend.


In [5]:
from keras.models import Sequential
from keras.layers import Conv1D,Dense,Dropout, BatchNormalization, Input, InputLayer,Activation, Flatten
from keras.activations import relu
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


In [6]:
model = Sequential()
model.add(InputLayer(input_shape=(500,31)))
model.add(Conv1D(filters=256, kernel_size=3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv1D(filters=128, kernel_size=2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv1D(filters=64, kernel_size=2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(7500))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(1500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(3,activation='softmax'))

model.compile(optimizer=Adam(lr=0.00001), metrics=['accuracy'], loss='categorical_crossentropy')

In [7]:
from keras.callbacks import ModelCheckpoint

In [9]:
checkpoint = ModelCheckpoint('my_model.h5', monitor='val_loss', verbose=1, save_best_only=True)


In [8]:
model.fit(x = data_set,
          y = to_categorical(targets),
          batch_size=256,
          epochs=20,
          validation_data = (val_data, to_categorical(val_targs))
          
         )

Train on 11369 samples, validate on 387 samples
Epoch 1/20
11369/11369 [==============================] - 43s 4ms/step - loss: 1.1930 - acc: 0.3927 - val_loss: 1.1673 - val_acc: 0.3643
Epoch 2/20
11369/11369 [==============================] - 13s 1ms/step - loss: 1.1168 - acc: 0.4398 - val_loss: 1.5102 - val_acc: 0.3101
Epoch 3/20
11369/11369 [==============================] - 10s 884us/step - loss: 1.0465 - acc: 0.5020 - val_loss: 2.5104 - val_acc: 0.3101
Epoch 4/20
11369/11369 [==============================] - 10s 885us/step - loss: 0.9564 - acc: 0.5541 - val_loss: 3.0716 - val_acc: 0.3049
Epoch 5/20
11369/11369 [==============================] - 10s 881us/step - loss: 0.8983 - acc: 0.5698 - val_loss: 5.7202 - val_acc: 0.3540
Epoch 6/20
11369/11369 [==============================] - 10s 884us/step - loss: 0.8318 - acc: 0.6025 - val_loss: 5.5917 - val_acc: 0.3566
Epoch 7/20
11369/11369 [==============================] - 10s 882us/step - loss: 0.8363 - acc: 0.5804 - val_loss: 3.2696 -